# LSA Model

In [31]:
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import os.path

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LsiModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # Author: don't skip this
# pyLDAvis.gensim.prepare

# I think i need another one:
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis

# Plots
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [32]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
stop_words.extend(['tenlastelegging\\n\\n\\naan', '\\n\\n\\n\\n\\n\\n', '\\n\\n\\n\\n\\n', '\\n\\n', 'een', 'naan', 'nde', 'nhij', '\\n'])

# Data

In [33]:
df = pd.read_csv('DataSample_DSP_07123.csv', index_col=1) 
df = df[df['re_tll'] != '[]'][['re_tll']] #re_bewijs
# data = [i for i in df['re_tll']]

# Tokenize 

In [34]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['re_tll']))

print(data_words[:1])

[['tenlastelegging', 'naan', 'verdachte', 'is', 'ten', 'laste', 'gelegd', 'dat', 'nhij', 'in', 'of', 'omstreeks', 'de', 'periode', 'van', 'november', 'tot', 'en', 'met', 'november', 'in', 'arnhem', 'en', 'of', 'in', 'nederland', 'en', 'of', 'in', 'duitsland', 'en', 'of', 'in', 'oostenrijk', 'en', 'of', 'in', 'hongarije', 'opzettelijk', 'een', 'of', 'meer', 'minderjarige', 'te', 'weten', 'naam', 'geboren', 'op', 'geboortedatum', 'in', 'geboorteplaats', 'en', 'of', 'naam', 'geboren', 'op', 'geboortedatum', 'in', 'geboorteplaats', 'heeft', 'onttrokken', 'aan', 'het', 'wettig', 'over', 'hen', 'haar', 'gesteld', 'gezag', 'en', 'of', 'aan', 'het', 'opzicht', 'van', 'degene', 'die', 'dit', 'desbevoegd', 'over', 'hen', 'haar', 'uitoefende', 'terwijl', 'die', 'minderjarige', 'beneden', 'de', 'twaalf', 'jaren', 'oud', 'was', 'waren', 'en', 'of', 'terwijl', 'list', 'was', 'gebezigd', 'bestaande', 'uit', 'het', 'door', 'verdachte', 'aan', 'de', 'school', 'van', 'de', 'minderjarig', 'in', 'strijd',

# Creating Bigram and Trigram Models

In [35]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['tenlastelegging', 'naan', 'verdachte', 'is', 'ten_laste_gelegd', 'dat', 'nhij', 'in', 'of', 'omstreeks', 'de', 'periode', 'van', 'november', 'tot', 'en', 'met', 'november', 'in', 'arnhem', 'en', 'of', 'in', 'nederland', 'en', 'of', 'in', 'duitsland', 'en', 'of', 'in', 'oostenrijk', 'en', 'of', 'in', 'hongarije', 'opzettelijk', 'een', 'of', 'meer', 'minderjarige', 'te', 'weten', 'naam', 'geboren', 'op', 'geboortedatum', 'in', 'geboorteplaats', 'en', 'of', 'naam', 'geboren', 'op', 'geboortedatum', 'in', 'geboorteplaats', 'heeft', 'onttrokken', 'aan', 'het', 'wettig', 'over', 'hen', 'haar', 'gesteld', 'gezag', 'en', 'of', 'aan', 'het', 'opzicht', 'van', 'degene', 'die', 'dit', 'desbevoegd', 'over', 'hen', 'haar', 'uitoefende', 'terwijl', 'die', 'minderjarige', 'beneden', 'de', 'twaalf_jaren', 'oud', 'was', 'waren', 'en', 'of', 'terwijl', 'list', 'was', 'gebezigd', 'bestaande', 'uit', 'het', 'door', 'verdachte', 'aan', 'de', 'school', 'van', 'de', 'minderjarig', 'in', 'strijd', 'met', 'd

# Remove Stopwords, Make Bigrams and Lemmatize

In [36]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [37]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('nl', disable=['parser', 'ner'])

nlp = spacy.load("nl_core_news_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['tenlastelegging', 'verdenken', 'laste_gelegd', 'periode', 'oostenrijk', 'opzettelijk', 'minderjarig', 'weten', 'naam', 'baren', 'geboortedatum', 'geboorteplaats', 'naam', 'baren', 'geboortedatum', 'geboorteplaats', 'onttrekken', 'wettig', 'stellen', 'gezag', 'opzicht', 'desbevoegd', 'uitoefenen', 'minderjarig', 'twaalf_jar', 'oud', 'list', 'bezigen', 'bestaan', 'verdenken', 'school', 'minderjarig', 'strijd', 'waarheid', 'mededeel', 'minderjarig', 'afspraak', 'tandarts', 'minderjarig', 'daarvoor', 'vroeger', 'halen', 'school', 'verdenken', 'huis', 'lok', 'verdachte', 'echtgenote', 'genaamd', 'naam', 'maken', 'afspraak', 'woonboulevard', 'beweerdelijk', 'gaan', 'shopp', 'verdenken', 'paspoort', 'kleding', 'toiletartikel', 'minderjarig', 'woning', 'toverweging', 'aanzien', 'bewijs', 'feit', 'nop', 'grond', 'bewijsmiddel', 'volgen', 'ver', 'discussie_staat', 'vaststellen', 'naangeefst', 'naam', 'verdenken', 'trouwen', 'samen', 'kind', 'naam', 'baren', 'geboortedatum', 'geboorteplaats', 

#  Create the Dictionary and Corpus needed for Topic Modeling

In [57]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 4), (1, 1), (2, 1), (3, 1), (4, 4), (5, 1), (6, 1), (7, 4), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 5), (32, 4), (33, 4), (34, 1), (35, 2), (36, 3), (37, 1), (38, 4), (39, 1), (40, 1), (41, 1), (42, 2), (43, 10), (44, 2), (45, 1), (46, 3), (47, 1), (48, 1), (49, 3), (50, 2), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 6), (58, 1), (59, 10), (60, 1), (61, 1), (62, 1), (63, 4), (64, 1), (65, 1), (66, 1), (67, 3), (68, 2), (69, 2), (70, 2), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 4), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 2), (95, 1), (96, 4), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 4), (104, 1), (105, 1), (106, 1), (107, 4), (108, 1), (109, 1), (110, 

In [58]:
# Id word
id2word[0]

'aangeefster'

# Building the Topic Model

In [59]:
from gensim.models import LsiModel
lsi_model = gensim.models.LsiModel(corpus=corpus,
                                   id2word=id2word,
                                   num_topics=5,
                                   chunksize=100, 
                                   power_iters = 10)
pprint(lsi_model.print_topics())
doc_lsi = lsi_model[corpus]

[(0,
  '0.582*"verdenken" + 0.303*"naam" + 0.213*"slachtoffer" + '
  '0.206*"medeverdacht" + 0.177*"kenteken" + 0.170*"uur" + 0.127*"adres" + '
  '0.126*"benadelen" + 0.120*"verklaren" + 0.116*"althans"'),
 (1,
  '0.400*"slachtoffer" + -0.344*"kenteken" + -0.303*"naam" + 0.270*"verdenken" '
  '+ -0.262*"uur" + -0.172*"aantreffen" + -0.172*"voertuig" + -0.165*"loods" + '
  '0.154*"althans" + -0.146*"stelen"'),
 (2,
  '-0.693*"slachtoffer" + 0.277*"betrokken" + -0.215*"naam" + '
  '0.208*"medeverdacht" + 0.182*"benadelen" + 0.180*"verdenken" + '
  '0.154*"adres" + -0.123*"kenteken" + 0.122*"man" + 0.112*"woning"'),
 (3,
  '-0.500*"naam" + -0.321*"benadelen" + 0.262*"slachtoffer" + '
  '0.234*"betrokken" + 0.184*"uur" + -0.175*"informatie" + 0.173*"adres" + '
  '0.149*"verbalisant" + 0.133*"man" + 0.130*"zien"'),
 (4,
  '-0.416*"benadelen" + 0.349*"naam" + 0.287*"medeverdacht" + -0.274*"bedrijf" '
  '+ 0.214*"slachtoffer" + -0.204*"althans" + 0.172*"informatie" + '
  '-0.162*"geval" + -0.

## Coherence score

In [60]:
# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score: ', coherence_lsi)


Coherence Score:  0.40395605159221726


## Improve LSA with TFIDF

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
#corpus = [str (item) for item in corpus]
from gensim import models

tfidf = models.TfidfModel(corpus)

In [65]:
corpus_tfidf = tfidf[corpus]

In [66]:
lsi_tfidf_model = gensim.models.LsiModel(corpus=corpus_tfidf,
                                   id2word=id2word,
                                   num_topics=5,
                                   chunksize=100, 
                                   power_iters = 10)
pprint(lsi_tfidf_model.print_topics())
doc_lsi = lsi_tfidf_model[corpus_tfidf]

[(0,
  '0.391*"slachtoffer" + 0.222*"naam" + 0.171*"geweld" + 0.164*"benadelen" + '
  '0.130*"althans" + 0.129*"ander" + 0.115*"geval" + 0.106*"gemeente" + '
  '0.105*"lichaam" + 0.102*"goed"'),
 (1,
  '-0.386*"slachtoffer" + 0.327*"benadelen" + 0.184*"partij" + '
  '-0.165*"feitelijk_weergegeven" + 0.151*"hoeveelheid" + '
  '-0.139*"verdenking_komt" + -0.139*"bijlage" + 0.135*"hennepplant" + '
  '0.133*"hennep" + 0.126*"adres"'),
 (2,
  '0.434*"benadelen" + 0.289*"partij" + -0.205*"feitelijk_weergegeven" + '
  '-0.179*"hoeveelheid" + -0.173*"hennepplant" + -0.170*"bijlage" + '
  '-0.162*"verdenking_komt" + -0.161*"hennep" + -0.142*"pand" + '
  '-0.139*"nfeit"'),
 (3,
  '0.208*"feitelijk_weergegeven" + 0.198*"aangeefster" + 0.197*"hof" + '
  '0.185*"bijlage" + 0.172*"aangever" + 0.158*"medeverdacht" + '
  '0.155*"verdenking_komt" + 0.128*"samen" + 0.128*"rechtbank" + '
  '0.123*"verklaren"'),
 (4,
  '-0.248*"benadelen" + -0.218*"partij" + 0.156*"bedrijf" + 0.153*"naam" + '
  '-0.152*"f

In [68]:
# Compute Coherence Score
coherence_model_lsi_tfidf = CoherenceModel(model=lsi_tfidf_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lsi_tfidf = coherence_model_lsi_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_lsi_tfidf)


Coherence Score:  0.5023381439633139


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(docs_rep[:,0], docs_rep[:,1], c=datax['Recommended IND'])
plt.title("Document Representation")
plt.show()